<a href="https://colab.research.google.com/github/CISC452MLProject/CISC-452-Group-22/blob/main/group22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from mmdet.apis import inference_detector, init_detector
import requests
from mmpose.utils import adapt_mmdet_pipeline
from mmpose.registry import VISUALIZERS
from mmpose.apis import inference_topdown
from mmpose.apis import init_model as init_pose_estimator
import mmcv
import numpy as np
from mmpose.evaluation.functional import nms
from mmpose.structures import merge_data_samples

In [2]:
dec_checkpoint_url = "https://download.openmmlab.com/mmpose/v1/projects/rtmpose/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth"
pos_config_url = "https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-body7_pt-body7_420e-256x192-e48f03d0_20230504.pth"

detector = init_detector(
        "mmpose/demo/mmdetection_cfg/rtmdet_m_640-8xb32_coco-person.py", dec_checkpoint_url, device="cuda")
detector.cfg = adapt_mmdet_pipeline(detector.cfg)


pose_estimator = init_pose_estimator(
        "mmpose/configs/body_2d_keypoint/rtmpose/body8/rtmpose-m_8xb256-420e_body8-256x192.py ",
        pos_config_url,
        device="cuda",
        cfg_options=dict(
            model=dict(test_cfg=dict(output_heatmaps=True))))

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmpose/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-body7_pt-body7_420e-256x192-e48f03d0_20230504.pth


In [3]:
pose_estimator.cfg.visualizer.radius = 3
pose_estimator.cfg.visualizer.alpha = 0.8
pose_estimator.cfg.visualizer.line_width = 1
visualizer = VISUALIZERS.build(pose_estimator.cfg.visualizer)

visualizer.set_dataset_meta(
    pose_estimator.dataset_meta, skeleton_style="mmpose")

In [4]:
img = "img/frame140.jpg"
det_result = inference_detector(detector, img)
pred_instance = det_result.pred_instances.cpu().numpy()
bboxes = np.concatenate(
    (pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
bboxes = bboxes[np.logical_and(pred_instance.labels == 0,
                                pred_instance.scores > 0.3)]
bboxes = bboxes[nms(bboxes, 0.3), :4]

c:\Users\mikeG\Documents\school\cisc-452\CISC-452-Group-22\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
pose_results = inference_topdown(pose_estimator, img, bboxes)
data_samples = merge_data_samples(pose_results)

c:\users\mikeg\documents\school\cisc-452\cisc-452-group-22\mmpose\mmpose\models\heads\coord_cls_heads\rtmcc_head.py:217: UserWarning: The predicted simcc values are normalized for visualization. This may cause discrepancy between the keypoint scores and the 1D heatmaps.
  warnings.warn('The predicted simcc values are normalized for '


In [6]:
img = mmcv.imread(img, channel_order='rgb')
visualizer.add_datasample(
            'result',
            img,
            data_sample=data_samples,
            draw_gt=False,
            draw_heatmap=True,
            draw_bbox=True,
            skeleton_style="mmpose",
            show=True)

array([[[ 31,  29,  50],
        [ 55,  53,  74],
        [ 59,  57,  78],
        ...,
        [ 13,  13,  13],
        [ 13,  13,  13],
        [ 13,  13,  13]],

       [[ 31,  29,  50],
        [ 55,  53,  74],
        [ 59,  57,  78],
        ...,
        [ 13,  13,  13],
        [ 13,  13,  13],
        [ 13,  13,  13]],

       [[ 30,  28,  49],
        [ 55,  53,  74],
        [ 59,  57,  78],
        ...,
        [ 13,  13,  13],
        [ 13,  13,  13],
        [ 13,  13,  13]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]